<a href="https://colab.research.google.com/github/doooooodlesssss/imagecap/blob/main/Copy_of_imgcap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# ⚠️ Restart runtime first, then run this cleanly:
!pip install "numpy<1.27" opencv-python==4.6.0.66 yolov4

# Patch the YOLOv4 weights.py
file_path = "/usr/local/lib/python3.12/dist-packages/yolov4/tf/utils/weights.py"
with open(file_path, "r") as f:
    code = f.read()

code = code.replace(
    "conv_shape = (filters, conv2d.input_shape[-1], *conv2d.kernel_size)",
    "conv_shape = (filters, conv2d.input.shape[-1], *conv2d.kernel_size)"
)

with open(file_path, "w") as f:
    f.write(code)

print("✅ Patched YOLOv4 weights.py")

# Import YOLOv4 directly after patch
from yolov4.tf import YOLOv4


✅ Patched YOLOv4 weights.py
Call tf.config.experimental.set_memory_growth(GPU0, True)


In [ ]:
repo_path = "/content/image_captioning"
%cd /content/drive/MyDrive/image_captioning/image_captioning/

kaggle_json = "/content/drive/MyDrive/image_captioning/kaggle.json"

dataset_dir = "/content/drive/MyDrive/image_captioning/datasets/coco2014"

feature_extraction_model = "xception"

model_dir = "trained_model_xception"

/content/drive/.shortcut-targets-by-id/1MRbY_LasFIeyhKUGSqG5utjF6qjx9rmq/image_captioning/image_captioning


In [ ]:
import tensorflow as tf
print("TensorFlow (preinstalled):", tf.__version__)
import os, importlib.util
import sys, pkgutil
print("Python:", sys.version)

def safe_pip(pkg, import_name=None):
    if import_name is None:
        import_name = pkg.split("==")[0]
    if importlib.util.find_spec(import_name) is None:
        !pip install {pkg}
    else:
        print(f"✔️ {pkg} already installed")

safe_pip("opencv-python-headless==4.6.0.66", "cv2")
safe_pip("matplotlib")
safe_pip("pillow", "PIL")
safe_pip("tqdm")
safe_pip("pycocotools")
safe_pip("lxml")
safe_pip("pandas")
safe_pip("seaborn")
safe_pip("yolov4")

if importlib.util.find_spec("tensorflow") is None:
    !pip install tensorflow
else:
    print("tensorflow already installed ✅")

if importlib.util.find_spec("kaggle") is None:
    !pip install kaggle
else:
    print("kaggle already installed ✅")



import numpy as np
import pickle
import matplotlib.pyplot as plt
from PIL import Image
import datetime
import importlib


from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import re

import time
import json
from glob import glob

from tqdm import tqdm
from termcolor import colored
from builtins import len

from timeit import default_timer as timer
import threading

import cv2

# import tensorflow_addons as tfa
import tensorflow_probability as tfp
from tensorflow.keras import layers, models





TensorFlow (preinstalled): 2.19.0
Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
✔️ opencv-python-headless==4.6.0.66 already installed
✔️ matplotlib already installed
✔️ pillow already installed
✔️ tqdm already installed
✔️ pycocotools already installed
✔️ lxml already installed
✔️ pandas already installed
✔️ seaborn already installed
✔️ yolov4 already installed
tensorflow already installed ✅
kaggle already installed ✅


In [ ]:
# hyperparameters
EMBEDDING_DIM = 256
MAX_LENGTH = 40
FEATURE_SHAPE = 2048   # CNN or YOLO feature dimension (flattened)

WORD_DICT_SIZE = 15000
LIMIT_SIZE = True
EXAMPLE_NUMBER = 20000  # will only work if LIMIT_SIZE is True
MY_EMBEDDING_DIM = 256
UNIT_COUNT = 512
MY_OPTIMIZER = tf.keras.optimizers.Adam()
MY_LOSS_OBJECT = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
EPOCH_COUNT = 20
REMOVE_CHECKPOINTS_AND_MODEL_AND_RETRAIN = True
DATASET = "mscoco"  # "mscoco" or "flickr8k" or "flickr30k"
TEST_SET_PROPORTION = 1
feature_extraction_model = "xception"
split = 0  # 0 for training, 1 for testing

BATCH_SIZE = 32  # 64
BUFFER_SIZE = 1000  # 1000
embedding_dim = MY_EMBEDDING_DIM  # hadie
units = UNIT_COUNT  # hadie
UNITS = UNIT_COUNT  # hadie
top_k = WORD_DICT_SIZE
vocab_size = top_k + 1
VOCAB_SIZE = top_k + 1


In [ ]:
# Python program for implementation of Quicksort Sort
def partition(arr, low, high):
	i = (low - 1)
	pivot = arr[high]
	for j in range(low, high):
		if arr[j][6] >= pivot[6]:
			i = i + 1
			arr[i], arr[j] = arr[j], arr[i]
	arr[i + 1], arr[high] = arr[high], arr[i + 1]
	return (i + 1)
def quickSort(arr, low, high):
	if len(arr) == 1:
		return arr
	if low < high:
		pi = partition(arr, low, high)
		quickSort(arr, low, pi - 1)
		quickSort(arr, pi + 1, high)

In [ ]:
sys.path.append("/content/drive/MyDrive/image_captioning/image_captioning")

In [ ]:
# class BahdanauAttention(tf.keras.Model):
#     def __init__(self, units):
#         super(BahdanauAttention, self).__init__()
#         self.W1 = layers.Dense(units)
#         self.W2 = layers.Dense(units)
#         self.V = layers.Dense(1)

#     def call(self, features, hidden):
#         # features: (batch_size, seq_len, embed_dim)
#         # hidden:   (batch_size, hidden_dim)
#         hidden_with_time_axis = tf.expand_dims(hidden, 1)
#         score = tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis))
#         attention_weights = tf.nn.softmax(self.V(score), axis=1)
#         context_vector = attention_weights * features
#         context_vector = tf.reduce_sum(context_vector, axis=1)
#         return context_vector, attention_weights


class BahdanauAttention(tf.keras.Model):
    def __init__(self, units):
        super().__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, features, hidden):
        hidden_time = tf.expand_dims(hidden, 1)
        score = tf.nn.tanh(self.W1(features) + self.W2(hidden_time))
        attn_weights = tf.nn.softmax(self.V(score), axis=1)
        context = attn_weights * features
        context = tf.reduce_sum(context, axis=1)
        return context, attn_weights



In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models


In [ ]:
# ====================================================
# 🔮 Spectral Attention Block (FFT-based)
# ====================================================
# class SpectralAttention(tf.keras.Model):
#     """
#     Applies learnable complex weighting in frequency domain.
#     Input: (B, D)
#     """
#     def __init__(self, feature_dim):
#         super().__init__()
#         # Learnable real and imaginary weights for each frequency
#         self.real_weight = tf.Variable(
#             tf.random.normal([feature_dim], stddev=0.02), trainable=True)
#         self.imag_weight = tf.Variable(
#             tf.random.normal([feature_dim], stddev=0.02), trainable=True)

#     def call(self, x):
#         # x: (B, D)
#         # 1D FFT along feature dimension
#         x_complex = tf.cast(x, tf.complex64)
#         freq = tf.signal.fft(x_complex)  # (B, D)

#         # combine learnable complex weights
#         spectral_weights = tf.complex(self.real_weight, self.imag_weight)
#         freq_weighted = freq * spectral_weights  # elementwise complex multiply

#         # inverse FFT to spatial domain
#         x_enriched = tf.math.real(tf.signal.ifft(freq_weighted))  # (B, D)
#         return x_enriched


class SpectralAttention(tf.keras.Model):
    def __init__(self, feature_dim):
        super().__init__()
        self.real_weight = tf.Variable(tf.random.normal([feature_dim], stddev=0.02))
        self.imag_weight = tf.Variable(tf.random.normal([feature_dim], stddev=0.02))

    def call(self, x):
        x_complex = tf.cast(x, tf.complex64)
        freq = tf.signal.fft(x_complex)
        spectral_weights = tf.complex(self.real_weight, self.imag_weight)
        weighted_freq = freq * spectral_weights
        x_out = tf.math.real(tf.signal.ifft(weighted_freq))
        return x_out



In [ ]:
# ====================================================
# ⚡ Dual Attention Fusion + Spectral Attention
# ====================================================
# class DualSpectralFusion(tf.keras.Model):
#     def __init__(self, units):
#         super().__init__()
#         self.att_yolo = BahdanauAttention(units)
#         self.att_xcep = BahdanauAttention(units)
#         self.spectral_att = SpectralAttention(units)

#     def call(self, yolo_feats, xcep_feats, hidden):
#         """
#         yolo_feats: (B, N_y, D)
#         xcep_feats: (B, N_x, D)
#         hidden:     (B, H)
#         """
#         yolo_ctx, _ = self.att_yolo(yolo_feats, hidden)  # (B, D)
#         xcep_ctx, _ = self.att_xcep(xcep_feats, hidden)  # (B, D)

#         fused = yolo_ctx * xcep_ctx  # elementwise fusion (global interaction)
#         enriched = self.spectral_att(fused)  # FFT attention

#         return enriched  # (B, D)


class DualSpectralFusion(tf.keras.Model):
    def __init__(self, units):
        super().__init__()
        self.att_yolo = BahdanauAttention(units)
        self.att_xcep = BahdanauAttention(units)
        self.spectral = SpectralAttention(units // 2)


    def call(self, yolo_feats, xcep_feats, hidden):
        yolo_ctx, _ = self.att_yolo(yolo_feats, hidden)
        xcep_ctx, _ = self.att_xcep(xcep_feats, hidden)
        fused = yolo_ctx * xcep_ctx
        enriched = self.spectral(fused)
        return enriched

In [ ]:
# =========================
# 🎛️ DCT/IDCT Utils
# =========================
def dct_2d(x):
    """Apply 2D Discrete Cosine Transform."""
    x = tf.signal.dct(x, type=2, norm='ortho')
    x = tf.signal.dct(tf.transpose(x, perm=[0, 2, 1]), type=2, norm='ortho')
    return tf.transpose(x, perm=[0, 2, 1])

def idct_2d(x):
    """Apply Inverse 2D DCT."""
    x = tf.signal.idct(x, type=2, norm='ortho')
    x = tf.signal.idct(tf.transpose(x, perm=[0, 2, 1]), type=2, norm='ortho')
    return tf.transpose(x, perm=[0, 2, 1])

def keep_low_freq(x, keep_ratio=0.25):
    """Keep only low-frequency components."""
    B, H, W = x.shape
    h_keep, w_keep = int(H * keep_ratio), int(W * keep_ratio)
    mask = np.zeros((H, W))
    mask[:h_keep, :w_keep] = 1.0
    mask = tf.constant(mask, dtype=tf.float32)
    return x * mask

In [ ]:
# =========================
# 🧩 Dual Attention Fusion Block
# # =========================
# class DualSpectralAttention(tf.keras.Model):
#     def __init__(self, units):
#         super(DualSpectralAttention, self).__init__()
#         self.att_yolo = BahdanauAttention(units)
#         self.att_xcep = BahdanauAttention(units)

#     def call(self, yolo_feats, xcep_feats, hidden):
#         """
#         yolo_feats: (batch_size, N_y, feature_dim)
#         xcep_feats: (batch_size, N_x, feature_dim)
#         hidden: (batch_size, hidden_dim)
#         """
#         # Apply separate Bahdanau attention
#         yolo_context, _ = self.att_yolo(yolo_feats, hidden)
#         xcep_context, _ = self.att_xcep(xcep_feats, hidden)

#         # Reshape to 2D (square-like) for DCT
#         side_len = int(np.sqrt(yolo_context.shape[-1]))
#         yolo_map = tf.reshape(yolo_context, (-1, side_len, side_len))
#         xcep_map = tf.reshape(xcep_context, (-1, side_len, side_len))

#         # Apply 2D DCT to both
#         yolo_freq = dct_2d(yolo_map)
#         xcep_freq = dct_2d(xcep_map)

#         # Keep only low frequencies
#         yolo_low = keep_low_freq(yolo_freq)
#         xcep_low = keep_low_freq(xcep_freq)

#         # Fuse via elementwise multiplication
#         fused_freq = yolo_low * xcep_low

#         # Return to spatial domain
#         fused_map = idct_2d(fused_freq)

#         # Flatten for decoder input
#         fused_flat = tf.reshape(fused_map, (-1, fused_map.shape[-1] * fused_map.shape[-2]))

#         return fused_flat


In [ ]:
# =========================
# 🧠 Decoder with DualSpectralAttention
# =========================
# class RNN_Decoder(tf.keras.Model):
#     def __init__(self, embedding_dim, units, vocab_size):
#         super(RNN_Decoder, self).__init__()
#         self.units = units
#         self.embedding = layers.Embedding(vocab_size, embedding_dim)
#         self.lstm = layers.LSTM(
#             self.units, return_sequences=True, return_state=True,
#             recurrent_initializer='glorot_uniform'
#         )
#         self.fc1 = layers.Dense(self.units)
#         self.fc2 = layers.Dense(vocab_size)

#         # Dual spectral attention block
#         self.dual_att = DualSpectralAttention(units)

#     def call(self, x, yolo_feats, xcep_feats, hidden):
#         fused_context = self.dual_att(yolo_feats, xcep_feats, hidden)

#         x = self.embedding(x)
#         x = tf.concat([tf.expand_dims(fused_context, 1), x], axis=-1)

#         output, state, _ = self.lstm(x)
#         x = self.fc1(output)
#         x = tf.reshape(x, (-1, x.shape[2]))
#         x = self.fc2(x)
#         return x, state

#     def reset_state(self, batch_size):
#         return tf.zeros((batch_size, self.units))


# ====================================================
# 🧠 Decoder with Spectral Attention Fusion
# ====================================================
# class RNN_Decoder(tf.keras.Model):
#     def __init__(self, embedding_dim, units, vocab_size):
#         super().__init__()
#         self.units = units
#         self.embedding = layers.Embedding(vocab_size, embedding_dim)
#         self.lstm = layers.LSTM(units, return_sequences=True, return_state=True)
#         self.fc1 = layers.Dense(units)
#         self.fc2 = layers.Dense(vocab_size)

#         self.dual_fusion = DualSpectralFusion(units)

#     def call(self, x, yolo_feats, xcep_feats, hidden):
#         # Apply dual-branch + spectral attention
#         fused_context = self.dual_fusion(yolo_feats, xcep_feats, hidden)  # (B, D)
#         x = self.embedding(x)  # (B, 1, D)
#         x = tf.concat([tf.expand_dims(fused_context, 1), x], axis=-1)  # (B,1,D+D)

#         output, state, _ = self.lstm(x)
#         x = self.fc1(output)  # (B,1,H)
#         x = tf.reshape(x, (-1, x.shape[2]))  # (B,H)
#         x = self.fc2(x)  # (B,vocab)
#         return x, state

#     def reset_state(self, batch_size):
#         return tf.zeros((batch_size, self.units))


class RNN_Decoder(tf.keras.Model):
    def __init__(self, embedding_dim, units, vocab_size):
        super().__init__()
        self.units = units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm = tf.keras.layers.LSTM(units, return_sequences=True, return_state=True)
        self.fc1 = tf.keras.layers.Dense(units)
        self.fc2 = tf.keras.layers.Dense(vocab_size)
        self.dual_fusion = DualSpectralFusion(units)

    def call(self, x, yolo_feats, xcep_feats, hidden):
        fused_context = self.dual_fusion(yolo_feats, xcep_feats, hidden)
        x = self.embedding(x)
        x = tf.concat([tf.expand_dims(fused_context, 1), x], axis=-1)
        output, state, _ = self.lstm(x)
        x = self.fc1(output)
        x = tf.reshape(x, (-1, x.shape[2]))
        x = self.fc2(x)
        return x, state

    def reset_state(self, batch_size):
        return tf.zeros((batch_size, self.units))

In [ ]:
# =========================
# 🧩 CNN Encoder (Xception) & YOLO Mock Encoders
# =========================
# class CNN_Encoder(tf.keras.Model):
#     def __init__(self, embedding_dim):
#         super(CNN_Encoder, self).__init__()
#         self.fc = layers.Dense(embedding_dim)

#     def call(self, x):
#         x = self.fc(x)
#         x = tf.nn.relu(x)
#         return x

class CNN_Encoder(tf.keras.Model):
    def __init__(self, embedding_dim):
        super().__init__()
        self.fc = layers.Dense(embedding_dim)

    def call(self, x):
        return tf.nn.relu(self.fc(x))


In [ ]:
# class YOLO_Encoder(tf.keras.Model):
#     def __init__(self, embedding_dim):
#         super(YOLO_Encoder, self).__init__()
#         self.fc = layers.Dense(embedding_dim)

#     def call(self, x):
#         x = self.fc(x)
#         x = tf.nn.relu(x)
#         return x


class YOLO_Encoder(tf.keras.Model):
    def __init__(self, embedding_dim):
        super().__init__()
        self.fc = layers.Dense(embedding_dim)

    def call(self, x):
        return tf.nn.relu(self.fc(x))


In [ ]:
# =========================
# 🚀 TRAINING PREPARATION
# =========================
encoder_xcep = CNN_Encoder(MY_EMBEDDING_DIM)
encoder_yolo = YOLO_Encoder(MY_EMBEDDING_DIM)
decoder = RNN_Decoder(MY_EMBEDDING_DIM, UNIT_COUNT, vocab_size)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')


# =========================
# 💡 Loss Function
# =========================
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)



In [ ]:
os.chdir(os.path.dirname(os.path.abspath("/content/drive/MyDrive/image_captioning/image_captioning")))
file = model_dir + "/max_length.txt"  # hadie
with open(file, 'r') as filetoread:  # hadie
    max_length = int(filetoread.readline())  # hadie

In [ ]:
# vocab_size = WORD_DICT_SIZE + 1
# encoder = CNN_Encoder(MY_EMBEDDING_DIM)
# decoder = RNN_Decoder(MY_EMBEDDING_DIM, UNIT_COUNT, vocab_size)
# decoder.build(input_shape=(None, max_length)) #diya

# decoder.load_weights(model_dir + "/my_model.weights.h5")


In [ ]:
# encoder_xcep = CNN_Encoder(MY_EMBEDDING_DIM)
# encoder_yolo = YOLO_Encoder(MY_EMBEDDING_DIM)
# decoder = RNN_Decoder(MY_EMBEDDING_DIM, UNIT_COUNT, vocab_size)

In [ ]:
mod = importlib.import_module("feature_extraction_model_" + feature_extraction_model)  # hadie
image_model = mod.image_model  # hadie
load_image = mod.load_image  # hadie
attention_features_shape = mod.attention_features_shape + 1  # hadie

new_input = image_model.input
hidden_layer = image_model.layers[-1].output

image_features_extract_model = tf.keras.Model(new_input, hidden_layer)

83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [ ]:
# loading the tokenizer
with open(model_dir + "/tokenizer.pickle", 'rb') as handle:
    tokenizer = pickle.load(handle)


def plot_attention(image, result, attention_plot):
    temp_image = np.array(Image.open(image))

    fig = plt.figure(figsize=(10, 10))

    len_result = len(result)
    for l in range(len_result):
        temp_att = np.resize(attention_plot[l], (8, 8))
        ax = fig.add_subplot(len_result // 2, len_result // 2, l + 1)
        ax.set_title(result[l])
        img = ax.imshow(temp_image)
        ax.imshow(temp_att, cmap='gray', alpha=0.6, extent=img.get_extent())

    plt.tight_layout()
    plt.show()

features_shape = 2048

In [ ]:
# def evaluate(image):
#     # attention_plot = np.zeros((max_length, attention_features_shape))
#     # hidden = decoder.reset_state(batch_size=1)
#     # temp_input = tf.expand_dims(load_image(image)[0], 0)
#     # img_tensor_val = image_features_extract_model(temp_input)
#     # img_tensor_val = tf.reshape(img_tensor_val, (img_tensor_val.shape[0], -1, img_tensor_val.shape[3]))
#     # yolo_features = image_path_to_yolo_bounding_boxes(image)  # hadie
#     # yolo_features = np.array(yolo_features.flatten())  # hadie
#     # yolo_features = np.pad(yolo_features, (0, features_shape - yolo_features.shape[0]), 'constant', constant_values=(0, 0)).astype(np.float32)  # hadie
#     # combined_features = np.vstack((img_tensor_val[0].numpy(), yolo_features)).astype(np.float32)  # hadie
#     # # features = encoder(combined_features)  # hadie
#     # half = img_tensor.shape[0] // 2
#     # xcep_feats = img_tensor[:half, :]
#     # yolo_feats = img_tensor[half:, :]
#     # xcep_encoded = encoder_xcep(xcep_feats)
#     # yolo_encoded = encoder_yolo(yolo_feats)
#     # predictions, hidden = decoder(dec_input, yolo_encoded, xcep_encoded, hidden)
#     # dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)

#     result = []

#     temp_input = tf.expand_dims(load_image(image)[0], 0)
#     img_tensor_val = image_features_extract_model(temp_input)
#     img_tensor_val = tf.reshape(img_tensor_val, (1, -1, img_tensor_val.shape[3]))

#     yolo_features = image_path_to_yolo_bounding_boxes(image)
#     yolo_features = np.pad(np.array(yolo_features).flatten(), (0, features_shape - len(yolo_features)), 'constant')
#     yolo_features = yolo_features.astype(np.float32)[None, :, None] # match dims
#     half = img_tensor_val.shape[1] // 2
#     xcep_feats = img_tensor_val[:, :half, :]
#     yolo_feats = tf.convert_to_tensor(yolo_features)[:, :half, :]

#     xcep_encoded = encoder_xcep(xcep_feats)
#     yolo_encoded = encoder_yolo(yolo_feats)

#     hidden = decoder.reset_state(batch_size=1)
#     dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)



#     for i in range(max_length):
#         predictions, hidden, attention_weights = decoder(dec_input, yolo_encoded, xcep_encoded, hidden)
#         attention_plot[i] = tf.reshape(attention_weights, (-1,)).numpy()

#         predicted_id = tf.random.categorical(predictions, 1)[0][0].numpy()

#         # Safe mapping: replace out-of-vocab token IDs with <unk> #diya
#         word = tokenizer.index_word.get(predicted_id, '<unk>')
#         result.append(word)
#         if word == '<end>':
#             # return result, attention_plot
#             break #diya


#         dec_input = tf.expand_dims([predicted_id], 0)

#     attention_plot = attention_plot[:len(result),:]
#     return result, attention_plot



In [ ]:
yolo = YOLOv4()

# yolo = YOLOv4(tiny=True)

yolo.config.parse_names("/content/drive/MyDrive/image_captioning/image_captioning/coco.names")
yolo.config.parse_cfg("/content/drive/MyDrive/image_captioning/image_captioning/yolov4.cfg")
# yolo.input_size = (480,640)

yolo.make_model()
yolo.load_weights("/content/drive/MyDrive/image_captioning/weights/yolov4.weights", weights_type="yolo")

# yolo.inference(media_path="C:/Users/Hadie/Desktop/yolo/NYC_14th_Street_looking_west_12_2005.jpg")


# the output is sorted according to the area by confidence
def image_path_to_yolo_bounding_boxes(image_path):  # , coco_dict, word_index):
    frame = cv2.imread(image_path)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    bboxes = yolo.predict(frame, prob_thresh=0.25)
    bboxes = bboxes.tolist()
    n = len(bboxes)
    # for each bounding box, append (area * confidence)
    for i in range(n):
        bboxes[i].append(bboxes[i][2] * bboxes[i][3] * bboxes[i][5])
        # obj_class_name = coco_dict[int(bboxes[i][4])].replace(" ", "")
        # if obj_class_name in word_index:
        #    bboxes[i][4] = word_index[coco_dict[int(bboxes[i][4])].replace(" ", "")]
        # else:
        #    bboxes[i][4] = word_index['<pad>']
    quickSort(bboxes, 0, n - 1)
    bboxes = np.array(bboxes)
    return bboxes


# raw feature extraction - not bounding boxes
def yolo_load_image(image_path):
    frame = cv2.imread(image_path)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # height, width, _ = frame.shape
    frame = yolo.resize_image(frame)
    frame = frame / 255.0
    frame = frame[np.newaxis, ...].astype(np.float32)
    return frame




/usr/local/lib/python3.12/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


main code:

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError:
        pass


tf.get_logger().setLevel('ERROR')  # hadie


os.chdir("/content/drive/MyDrive/image_captioning/image_captioning")
sys.path.append(os.getcwd())  # add current folder to Python path

os.chdir(os.path.dirname(os.path.abspath("/content/drive/MyDrive/image_captioning/image_captioning")))  # hadie

from yolo import image_path_to_yolo_bounding_boxes  # hadie
start_date = datetime.datetime.now()  # hadie
my_start = timer()  # hadie

if not os.path.exists("trained_model_" + feature_extraction_model):  # create the dicrectory if it does not exists # hadie
    os.makedirs("trained_model_" + feature_extraction_model)  # hadie


In [ ]:
# Annotation folder path
annotation_folder = "/content/drive/MyDrive/image_captioning/datasets/coco2014/annotations/"

# Pick the right annotation file
if split == 0:  # training split
    annotation_file = os.path.join(annotation_folder, "captions_train2014.json")
    image_folder = "/content/drive/MyDrive/image_captioning/datasets/coco2014/train2014/"

else:           # validation split
    annotation_file = os.path.join(annotation_folder, "captions_val2014.json")
    image_folder = "/content/drive/MyDrive/image_captioning/datasets/coco2014/val2014/"

PATH = image_folder


In [ ]:
with open(annotation_file, 'r') as f:
    annotations = json.load(f)



all_captions = []
all_img_name_vector = []
all_ids = []  # hadie

image_id_index = {}  # hadie
for img in annotations['images']:  # hadie
    image_id_index[img['id']] = img['file_name']  # hadie

for annot in annotations['annotations']:
    caption = '<start> ' + annot['caption'] + ' <end>'
    image_id = annot['image_id']
    if DATASET == "mscoco":  # hadie
        full_coco_image_path = PATH + image_id_index[image_id]
        # print(full_coco_image_path, image_id)
    else:  # hadie
        full_coco_image_path = PATH + image_id + ".jpg"  # hadie
    all_ids.append(image_id)  # hadie
    all_img_name_vector.append(full_coco_image_path)
    all_captions.append(caption)

train_ids, train_captions, img_name_vector = shuffle(all_ids,  # hadie
                                         all_captions,
                                         all_img_name_vector,
                                         random_state=1)

# Select the first 30000 captions from the shuffled set
num_examples = EXAMPLE_NUMBER  # hadie
if LIMIT_SIZE:  # hadie
    train_ids = train_ids[:num_examples]
    train_captions = train_captions[:num_examples]
    img_name_vector = img_name_vector[:num_examples]

print("training captions: ", len(train_captions), ", all captions: ", len(all_captions))  # hadie

training captions:  20000 , all captions:  414113


In [ ]:
mod = importlib.import_module("feature_extraction_model_" + feature_extraction_model)  # hadie
image_model = mod.image_model  # hadie
load_image = mod.load_image  # hadie
attention_features_shape = mod.attention_features_shape + 1  # hadie
new_input = image_model.input
hidden_layer = image_model.layers[-1].output

image_features_extract_model = tf.keras.Model(new_input, hidden_layer)

In [ ]:
print("-----------------------------START OF EXECUTION-----------------------------")

# Get unique images
encode_train = sorted(set(img_name_vector))

# Only keep unprocessed images
encode_train = [
    x for x in encode_train
    if not os.path.exists(x + "_" + feature_extraction_model + ".npy")
]

# ✅ Filter out missing files
encode_train = [x for x in encode_train if os.path.exists(x)]

print(f"extracting features ({len(encode_train)}) valid file(s)")  # hadie

if len(encode_train) > 0:
    image_dataset = tf.data.Dataset.from_tensor_slices(encode_train)

    # --- Safe loader that skips missing files ---
    def safe_load_image(path):
        try:
            img = tf.io.read_file(path)
            img = tf.image.decode_jpeg(img, channels=3)
            img = tf.image.resize(img, (299, 299))
            img = tf.keras.applications.inception_v3.preprocess_input(img)
            return img, path
        except tf.errors.NotFoundError:
            # Skip missing or unreadable images
            print(f"⚠️ Skipping missing file: {path.numpy().decode('utf-8')}")
            return None

    def filter_none(data):
        return data is not None

    # Map safely
    image_dataset = (
        image_dataset
        .map(lambda path: tf.py_function(safe_load_image, [path], [tf.float32, tf.string]),
             num_parallel_calls=tf.data.experimental.AUTOTUNE)
        .filter(lambda img, path: tf.reduce_all(tf.not_equal(tf.size(img), 0)))  # ensure valid tensors
        .batch(16)
    )

    for img, path in tqdm(image_dataset):
        batch_features = image_features_extract_model(img)
        batch_features = tf.reshape(batch_features, (batch_features.shape[0], -1, batch_features.shape[3]))

        for bf, p in zip(batch_features, path):
            path_of_feature = p.numpy().decode("utf-8")

            # --- YOLO + combined features ---
            yolo_features = image_path_to_yolo_bounding_boxes(path_of_feature)
            yolo_features = np.array(yolo_features).flatten()
            yolo_features = np.pad(yolo_features, (0, features_shape - len(yolo_features)),
                                   'constant', constant_values=(0, 0)).astype(np.float32)

            combined_features = np.vstack((bf.numpy(), yolo_features)).astype(np.float32)
            np.save(path_of_feature + "_" + feature_extraction_model, combined_features)

print("✅ finished extracting features")  # hadie


-----------------------------START OF EXECUTION-----------------------------
extracting features (0) valid file(s)
✅ finished extracting features


In [ ]:
# Find the maximum length of any caption in our dataset
def calc_max_length(tensor):
    return max(len(t) for t in tensor)


# Choose the top 5000 words from the vocabulary
top_k = WORD_DICT_SIZE  # hadie

if not REMOVE_CHECKPOINTS_AND_MODEL_AND_RETRAIN:  # hadie
    print("using the cashed tokenizer")  # hadie
    # loading the tokenizer # hadie
    with open("trained_model_" + feature_extraction_model + "/tokenizer.pickle", 'rb') as handle:  # hadie
        tokenizer = pickle.load(handle)  # hadie
else:  # hadie
    print("tokenizing and padding captions")  # hadie
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=top_k,
                                                      oov_token="<unk>",
                                                      filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
    tokenizer.fit_on_texts(train_captions)
    train_seqs = tokenizer.texts_to_sequences(train_captions)  # 777 maybe this line needs removal
    tokenizer.word_index['<pad>'] = 0
    tokenizer.index_word[0] = '<pad>'
    # saving the tokenizer to disk # hadie
    with open("trained_model_" + feature_extraction_model + "/tokenizer.pickle", 'wb') as handle:  # hadie
        pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)  # hadie

# Create the tokenized vectors
train_seqs = tokenizer.texts_to_sequences(train_captions)

# Pad each vector to the max_length of the captions
# If you do not provide a max_length value, pad_sequences calculates it automatically
cap_vector = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, padding='post')

if not REMOVE_CHECKPOINTS_AND_MODEL_AND_RETRAIN:  # hadie
    file = "trained_model_" + feature_extraction_model + "/max_length.txt"  # hadie
    with open(file, 'r') as filetoread:  # hadie
        max_length = int(filetoread.readline())  # hadie
else:  # hadie
    # Calculates the max_length, which is used to store the attention weights
    max_length = calc_max_length(train_seqs)

    file = "trained_model_" + feature_extraction_model + "/max_length.txt"  # hadie
    with open(file, 'w') as filetowrite:  # hadie
        filetowrite.write(str(max_length))  # write the maximum length to disk # hadie

print("finished tokenizing and padding captions")  # hadie

"""## Split the data into training and testing"""

# # Create training and validation sets using an 80-20 split

# img_name_train, img_name_val, cap_train, cap_val = train_test_split(
#     img_name_vector, cap_vector, test_size=0.2, random_state=42
# )


# new spli w image_id_val
image_ids = np.arange(len(img_name_vector))  # assign IDs before splitting

image_id_train, image_id_val, img_name_train, img_name_val, cap_train, cap_val = train_test_split(
    image_ids, img_name_vector, cap_vector, test_size=0.2, random_state=42
)



<>:18: SyntaxWarning: invalid escape sequence '\]'
<>:18: SyntaxWarning: invalid escape sequence '\]'
/tmp/ipython-input-1048036035.py:18: SyntaxWarning: invalid escape sequence '\]'
  filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')


tokenizing and padding captions
finished tokenizing and padding captions


In [ ]:
print("len(img_name_train) = ", len(img_name_train), ", len(cap_train) = ", len(cap_train), ", len(img_name_val) = ", len(img_name_val), ", len(cap_val) = ", len(cap_val))  # hadie

len(img_name_train) =  16000 , len(cap_train) =  16000 , len(img_name_val) =  4000 , len(cap_val) =  4000


In [ ]:
num_steps = len(img_name_train) // BATCH_SIZE

def map_func(img_name, cap):
  img_tensor = np.load(img_name.decode('utf-8') + "_" + feature_extraction_model + '.npy')
  return img_tensor, cap


dataset = tf.data.Dataset.from_tensor_slices((img_name_train, cap_train))

# Use map to load the numpy files in parallel
dataset = dataset.map(lambda item1, item2: tf.numpy_function(
          map_func, [item1, item2], [tf.float32, tf.int32]),
          num_parallel_calls=tf.data.experimental.AUTOTUNE)

# Shuffle and batch
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [ ]:

optimizer = MY_OPTIMIZER  # hadie
loss_object = MY_LOSS_OBJECT  # hadie


def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [ ]:
checkpoint_path = "./checkpoints/train"
# hadie
if REMOVE_CHECKPOINTS_AND_MODEL_AND_RETRAIN:
    try:
        for filename in os.listdir(checkpoint_path):
            print("deleting " + checkpoint_path + "/" + filename)
            os.unlink(checkpoint_path + "/" + filename)
    except Exception as e:
        # print('Failed to delete %s. Reason: %s' % (checkpoint_path + "/" + filename, e))
        print("Failed to delete checkpoint(s). Reason:", e)
    # remove the saved model too
    if os.path.exists("./trained_model_" + feature_extraction_model + "/my_model.index"):
        print("deleting trained_model_" + feature_extraction_model + "/my_model.index")
        os.unlink("./trained_model_" + feature_extraction_model + "/my_model.index")
    if os.path.exists("./trained_model_" + feature_extraction_model + "/checkpoint"):
        print("deleting /trained_model_" + feature_extraction_model + "/checkpoint")
        os.unlink("./trained_model_" + feature_extraction_model + "/checkpoint")
    if os.path.exists("./trained_model_" + feature_extraction_model + "/my_model.data-00000-of-00001"):
        print("deleting trained_model_" + feature_extraction_model + "/my_model.data-00000-of-00001")
        os.unlink("./trained_model_" + feature_extraction_model + "/my_model.data-00000-of-00001")
    if os.path.exists("./trained_model_" + feature_extraction_model + "/learning_curve.png"):
        print("deleting trained_model_" + feature_extraction_model + "/learning_curve.png")
        os.unlink("./trained_model_" + feature_extraction_model + "/learning_curve.png")

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001) #diya

ckpt = tf.train.Checkpoint(encoder_xcep=encoder_xcep,
                           encoder_yolo=encoder_yolo,
                           decoder=decoder,
                           optimizer=optimizer)


start_epoch = 0


ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)
if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("✅ Restored dual spectral checkpoint")



In [ ]:
print("🧹 Cleaning dataset — skipping missing .npy feature files...")

# Filter all valid feature files before creating dataset
valid_img_paths = []
missing_count = 0

for path in img_name_vector:
    feature_path = path + "_" + feature_extraction_model + ".npy"
    if os.path.exists(feature_path):
        valid_img_paths.append(path)
    else:
        missing_count += 1

print(f"✅ Found {len(valid_img_paths)} valid images.")
print(f"⚠️ Skipped {missing_count} missing feature files.\n")

# Build dataset from only valid files
img_name_train = valid_img_paths
cap_train = [cap for img, cap in zip(img_name_vector, cap_vector) if img in valid_img_paths]

# Now recreate dataset safely
def map_func(img_name, cap):
    # Decode bytes to string if needed
    img_path = img_name.numpy().decode('utf-8')
    feature_path = img_path + "_" + feature_extraction_model + ".npy"
    # guaranteed to exist from filtering above
    img_tensor = np.load(feature_path)
    return img_tensor, cap

def tf_map_func(img_name, cap):
    return tf.py_function(map_func, [img_name, cap], [tf.float32, tf.int32])

dataset = tf.data.Dataset.from_tensor_slices((img_name_train, cap_train))
dataset = dataset.shuffle(buffer_size=1000)
dataset = dataset.map(tf_map_func, num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = dataset.batch(64)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

print("✅ Dataset built successfully — all missing .npy files skipped.")


🧹 Cleaning dataset — skipping missing .npy feature files...
✅ Found 11987 valid images.
⚠️ Skipped 8013 missing feature files.

✅ Dataset built successfully — all missing .npy files skipped.


In [ ]:
# ============================================================
# 🚀 Spectral Attention Captioning — Training + Evaluation
# ============================================================

import tensorflow as tf
import numpy as np
from tqdm import tqdm
import os, json, time

# ------------------------------------------------------------
# 1️⃣ Model Components (already defined above)
# ------------------------------------------------------------
encoder_xcep = CNN_Encoder(MY_EMBEDDING_DIM)
encoder_yolo = YOLO_Encoder(MY_EMBEDDING_DIM)
decoder = RNN_Decoder(MY_EMBEDDING_DIM, UNIT_COUNT, vocab_size)

optimizer = tf.keras.optimizers.Adam(1e-3)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

# ------------------------------------------------------------
# 2️⃣ Loss Function
# ------------------------------------------------------------
@tf.function
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)

# ------------------------------------------------------------
# 3️⃣ Train Step (Spectral Fusion)
# ------------------------------------------------------------
@tf.function
def train_step(img_tensor, target):
    """
    img_tensor: loaded combined npy feature (Xception+YOLO)
    target: padded caption tensor
    """
    loss = 0
    hidden = decoder.reset_state(batch_size=target.shape[0])
    dec_input = tf.expand_dims([tokenizer.word_index['<start>']] * target.shape[0], 1)

    # Split combined features into Xception + YOLO halves
    seq_len = img_tensor.shape[1] // 2
    xcep_feats = img_tensor[:, :seq_len, :]
    yolo_feats = img_tensor[:, seq_len:, :]

    with tf.GradientTape() as tape:
        xcep_encoded = encoder_xcep(xcep_feats)
        yolo_encoded = encoder_yolo(yolo_feats)

        for i in range(1, target.shape[1]):
            preds, hidden = decoder(dec_input, yolo_encoded, xcep_encoded, hidden)
            loss += loss_function(target[:, i], preds)
            dec_input = tf.expand_dims(target[:, i], 1)

    total_loss = loss / int(target.shape[1])

    vars_all = (
        encoder_xcep.trainable_variables +
        encoder_yolo.trainable_variables +
        decoder.trainable_variables
    )
    grads = tape.gradient(loss, vars_all)
    optimizer.apply_gradients(zip(grads, vars_all))
    return total_loss

# ------------------------------------------------------------
# 4️⃣ Dataset Loader — from your .npy files
# ------------------------------------------------------------
def map_func(img_name, cap):
    feature_path = img_name.numpy().decode('utf-8') + "_" + feature_extraction_model + ".npy"
    img_tensor = np.load(feature_path)
    img_tensor = tf.convert_to_tensor(img_tensor, dtype=tf.float32)
    return img_tensor, cap

def tf_map_func(img_name, cap):
    return tf.py_function(map_func, [img_name, cap], [tf.float32, tf.int32])

dataset = tf.data.Dataset.from_tensor_slices((img_name_train, cap_train))
dataset = dataset.shuffle(1000)
dataset = dataset.map(tf_map_func, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# ------------------------------------------------------------
# 5️⃣ Checkpointing
# ------------------------------------------------------------
checkpoint_dir = "./checkpoints_spectral"
ckpt = tf.train.Checkpoint(
    encoder_xcep=encoder_xcep,
    encoder_yolo=encoder_yolo,
    decoder=decoder,
    optimizer=optimizer
)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_dir, max_to_keep=5)

if REMOVE_CHECKPOINTS_AND_MODEL_AND_RETRAIN:
    if os.path.exists(checkpoint_dir):
        for f in os.listdir(checkpoint_dir):
            os.remove(os.path.join(checkpoint_dir, f))
    print("🧹 Cleared old checkpoints")

# ------------------------------------------------------------
# 6️⃣ Training Loop
# ------------------------------------------------------------
EPOCHS = 20
loss_plot = []

for epoch in range(EPOCHS):
    start = time.time()
    total_loss = 0
    print(f"\n🧠 Epoch {epoch+1}/{EPOCHS}")

    for batch, (img_tensor, target) in enumerate(tqdm(dataset)):
        batch_loss = train_step(img_tensor, target)
        total_loss += batch_loss

        if batch % 100 == 0:
            print(f"Batch {batch} | Loss: {batch_loss.numpy():.4f}")

    avg_loss = total_loss / len(dataset)
    loss_plot.append(float(avg_loss))
    ckpt_manager.save()
    print(f"✅ Epoch {epoch+1} completed | Avg Loss: {avg_loss:.4f} | Time: {time.time()-start:.2f}s")

# ------------------------------------------------------------
# 7️⃣ Save Model + Tokenizer + Training History
# ------------------------------------------------------------
save_dir = f"/content/drive/MyDrive/image_captioning/trained_model_spectral"
os.makedirs(save_dir, exist_ok=True)

encoder_xcep.save_weights(os.path.join(save_dir, "encoder_xcep.weights.h5"))
encoder_yolo.save_weights(os.path.join(save_dir, "encoder_yolo.weights.h5"))
decoder.save_weights(os.path.join(save_dir, "decoder.weights.h5"))

with open(os.path.join(save_dir, "loss_plot.json"), "w") as f:
    json.dump(loss_plot, f)

with open(os.path.join(save_dir, "tokenizer.pkl"), "wb") as f:
    pickle.dump(tokenizer, f)

print("✅ Training complete. Models saved to:", save_dir)

# ------------------------------------------------------------
# 8️⃣ Evaluation (Generate captions)
# ------------------------------------------------------------
def evaluate_image(image_path):
    """Generate a caption for one image using spectral attention decoder."""
    img_tensor = np.load(image_path + "_" + feature_extraction_model + ".npy")
    img_tensor = tf.expand_dims(img_tensor, 0)

    seq_len = img_tensor.shape[1] // 2
    xcep_feats = img_tensor[:, :seq_len, :]
    yolo_feats = img_tensor[:, seq_len:, :]

    xcep_encoded = encoder_xcep(xcep_feats)
    yolo_encoded = encoder_yolo(yolo_feats)

    hidden = decoder.reset_state(batch_size=1)
    dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)

    result = []
    for i in range(max_length):
        predictions, hidden = decoder(dec_input, yolo_encoded, xcep_encoded, hidden)
        predicted_id = tf.argmax(predictions[0]).numpy()
        word = tokenizer.index_word.get(predicted_id, '')
        if word == '<end>':
            break
        result.append(word)
        dec_input = tf.expand_dims([predicted_id], 0)

    return ' '.join(result)

# ------------------------------------------------------------
# 9️⃣ Run Evaluation on Validation Subset
# ------------------------------------------------------------
val_results = []
sample_imgs = img_name_val[:500]  # evaluate on 500 images

for img_path in tqdm(sample_imgs):
    if not os.path.exists(img_path + "_" + feature_extraction_model + ".npy"):
        continue
    caption = evaluate_image(img_path)
    val_results.append({
        "image_id": os.path.basename(img_path),
        "predicted_caption": caption
    })

results_path = os.path.join(save_dir, "results_val.json")
with open(results_path, "w") as f:
    json.dump(val_results, f, indent=2)

print(f"✅ Saved {len(val_results)} validation captions → {results_path}")


🧹 Cleared old checkpoints

🧠 Epoch 1/20


  0%|          | 1/375 [01:07<7:02:19, 67.75s/it]

Batch 0 | Loss: 2.2507


 27%|██▋       | 101/375 [28:58<1:19:52, 17.49s/it]

Batch 100 | Loss: 1.1902


...

Batch 300 | Loss: 0.2545


100%|██████████| 375/375 [06:04<00:00,  1.03it/s]


✅ Epoch 20 completed | Avg Loss: 0.2442 | Time: 365.57s
✅ Training complete. Models saved to: /content/drive/MyDrive/image_captioning/trained_model_spectral


100%|██████████| 500/500 [03:44<00:00,  2.23it/s]

✅ Saved 311 validation captions → /content/drive/MyDrive/image_captioning/trained_model_spectral/results_val.json


In [ ]:
# loss_plot = []

# @tf.function
# def train_step(img_tensor, target):
#     loss = 0
#     hidden = decoder.reset_state(batch_size=target.shape[0])
#     dec_input = tf.expand_dims([tokenizer.word_index['<start>']] * target.shape[0], 1)

#     # Split the .npy file into YOLO + Xception sub-features
#     half = img_tensor.shape[1] // 2
#     xcep_feats = img_tensor[:, :half, :]
#     yolo_feats = img_tensor[:, half:, :]

#     with tf.GradientTape() as tape:
#         xcep_encoded = encoder_xcep(xcep_feats)
#         yolo_encoded = encoder_yolo(yolo_feats)
#         for i in range(1, target.shape[1]):
#             predictions, hidden = decoder(dec_input, yolo_encoded, xcep_encoded, hidden)
#             loss += loss_function(target[:, i], predictions)
#             dec_input = tf.expand_dims(target[:, i], 1)

#     total_loss = loss / int(target.shape[1])
#     variables = (encoder_yolo.trainable_variables +
#                  encoder_xcep.trainable_variables +
#                  decoder.trainable_variables)
#     gradients = tape.gradient(loss, variables)
#     optimizer.apply_gradients(zip(gradients, variables))

#     return loss, total_loss


# EPOCHS = 20  # hadie

# # 🧠 Helper to safely load batches
# def safe_batch_loader(dataset):
#     """Yield batches, skipping missing files gracefully."""
#     for batch, (img_tensor, target) in enumerate(dataset):
#         try:
#             yield batch, (img_tensor, target)
#         except (FileNotFoundError, tf.errors.NotFoundError) as e:
#             print(f"⚠️ Skipping missing or corrupted file in batch {batch}: {e}")
#             continue


# if not os.path.exists("trained_model_" + feature_extraction_model + "/my_model.index"):  # hadie
#     print("training..")  # hadie
#     for epoch in range(start_epoch, EPOCHS):
#         print(f"Starting epoch {epoch+1}/{EPOCHS}")
#         start = time.time()
#         total_loss = 0

#         for batch, (img_tensor, target) in safe_batch_loader(dataset):
#             try:
#                 batch_loss, t_loss = train_step(img_tensor, target)
#                 total_loss += t_loss
#             except (FileNotFoundError, tf.errors.NotFoundError) as e:
#                 print(f"⚠️ Skipping batch {batch} due to missing file: {e}")
#                 continue

#             if batch % 100 == 0:
#                 print(f"Epoch {epoch+1} Batch {batch} Loss {batch_loss.numpy():.4f}")
#                 print('Epoch {} Batch {} Loss {:.4f}'.format(
#                     epoch + 1, batch, batch_loss.numpy() / int(target.shape[1])))

#         if len(dataset) > 0:
#             avg_loss = total_loss / len(dataset)
#         else:
#             avg_loss = 0

#         print(f"Epoch {epoch+1} Loss {avg_loss:.4f}")
#         loss_plot.append(avg_loss)

#         if epoch % 5 == 0:
#             ckpt_manager.save()

#         print(f"Epoch {epoch+1} Loss {avg_loss:.6f}")
#         print(f"Time taken for 1 epoch {time.time() - start:.2f} sec\n")

#     # Save model
#     save_dir = f"trained_model_{feature_extraction_model}"

#     base_dir = "/content/drive/MyDrive/image_captioning"

#     save_dir = os.path.join(base_dir, f"trained_model_{feature_extraction_model}")

#     os.makedirs(save_dir, exist_ok=True)
#     decoder.save_weights(os.path.join(save_dir, "my_model.weights.h5"))

# else:  # hadie
#     print("A trained model has been found. Loading it from disk..")  # hadie
#     decoder.load_weights("trained_model_" + feature_extraction_model + "/my_model.weights.h5")


In [ ]:
# encoder_xcep.save_weights(os.path.join(save_dir, "encoder_xcep.weights.h5"))
# encoder_yolo.save_weights(os.path.join(save_dir, "encoder_yolo.weights.h5"))
# decoder.save_weights(os.path.join(save_dir, "decoder.weights.h5"))


In [ ]:
# with open(os.path.join(save_dir, "loss_plot.json"), "w") as f:
#     json.dump([float(l) for l in loss_plot], f)

In [ ]:
# with open(os.path.join(save_dir, "tokenizer.pkl"), "wb") as f:
#     pickle.dump(tokenizer, f)

In [ ]:
# #load loss and tokeniser
# save_dir = f"trained_model_{feature_extraction_model}"

# with open(os.path.join(save_dir, "loss_plot.json")) as f:
#     loss_plot = json.load(f)

# with open(os.path.join(save_dir, "tokenizer.pkl"), "rb") as f:
#     tokenizer = pickle.load(f)


In [ ]:
# def plot_attention(image, result, attention_plot):
#     temp_image = np.array(Image.open(image))

#     fig = plt.figure(figsize=(10, 10))

#     len_result = len(result)
#     for l in range(len_result):
#         temp_att = np.resize(attention_plot[l], (8, 8))
#         ax = fig.add_subplot(len_result // 2, len_result // 2, l + 1)
#         ax.set_title(result[l])
#         img = ax.imshow(temp_image)
#         ax.imshow(temp_att, cmap='gray', alpha=0.6, extent=img.get_extent())

#     plt.tight_layout()
#     plt.show()

In [ ]:
# print("Total img_name_val:", len(img_name_val))
# print("Example path:", img_name_val[0])

In [ ]:
# def validate_image(id, image_name_for_validation, original_caption):
#   try:
#     print("Evaluating:", image_name_for_validation.decode("utf-8"), "From thread", threading.current_thread().ident)

#     result, attention_plot = evaluate(image_name_for_validation)  # generate the hypothesis
#     result = ' '.join(result).replace("<end>", "").strip()  # remove unnecessary characters

#     dict = {}
#     dict["image_id"] = int(id)  # .decode("utf-8")
#     dict["caption"] = result;
#     dict["original_caption"] = original_caption.decode("utf-8")
#     dict["file_name"] = image_name_for_validation.decode("utf-8")
#     return json.dumps(dict)
#   except Exception as e:
#         print("⚠️ Skipping image:", image_name_for_validation.decode("utf-8"), "Error:", e)
#         return json.dumps({})  # return an empty dict so the pipeline doesn't break

# caption_strings_val = list(map(lambda item: (' '.join([tokenizer.index_word[i] for i in item if i not in [0]])).replace("<end>", "").replace("<start>", "").strip(), cap_val))  # convert to a list of strings
# validation_dataset = tf.data.Dataset.from_tensor_slices((image_id_val, img_name_val, caption_strings_val))  # create the dataset
# validation_dataset = validation_dataset.take(300)  # only evaluate first 100 images
# eval_start_date = datetime.datetime.now()  # hadie
# list_of_dicts = validation_dataset.map(lambda item1, item2, item3: tf.numpy_function(validate_image, [item1, item2, item3], [tf.string]), num_parallel_calls=1)  # run in parallel
# list_of_dicts = list(list_of_dicts.as_numpy_iterator())  # convert to a list
# list_of_dicts = [item for sublist in list_of_dicts for item in sublist]  # flatten the list by removing nested tuples
# list_of_dicts = list(map(lambda item: json.loads(item), list_of_dicts))  # rewrap the strings as dictionaries, then convert to a list

# added_ids = []

# # remove empty dicts (those from skipped images)
# list_of_dicts = [d for d in list_of_dicts if "image_id" in d]

# unique_list_of_dicts = []
# for d in list_of_dicts:
#     if d["image_id"] not in added_ids:
#         added_ids.append(d["image_id"])
#         unique_list_of_dicts.append(d)

# file = "trained_model_" + feature_extraction_model + "/results.json"  # hadie
# with open(file, 'w') as filetowrite:  # hadie
#     filetowrite.write(json.dumps(unique_list_of_dicts))  # hadie

# print("The results have been written to trained_model_" + feature_extraction_model + "/results.json")  # hadie
# print("Main thread:", threading.current_thread().ident)  # hadie

# end_date = datetime.datetime.now()  # hadie

# my_end = timer()  # hadie
# hours, rem = divmod(my_end - my_start, 3600)  # hadie
# minutes, seconds = divmod(rem, 60)  # hadie

# print("Start time: " + str(start_date))  # hadie
# print("Evaluation start time: " + str(eval_start_date))  # hadie
# print("End time: " + str(end_date))  # hadie

# print("Time elapsed (hours:minutes:seconds): {:0>2}:{:0>2}:{:05.2f}".format(int(hours), int(minutes), seconds))  # hadie

# if REMOVE_CHECKPOINTS_AND_MODEL_AND_RETRAIN:  # hadie
#     plt.plot(loss_plot)
#     plt.xlabel('Epochs')
#     plt.ylabel('Loss')
#     plt.title('Loss Plot')
#     plt.savefig("trained_model_" + feature_extraction_model + "/learning_curve.png")  # hadie
#     # plt.show() # hadie

# print("The learning curve has been written to trained_model_" + feature_extraction_model + "/learning_curve.png")  # hadie
# print("Feature extraction model: " + feature_extraction_model)
# print("Dataset: " + DATASET)
# print("Development set proportion: " + str(TEST_SET_PROPORTION))
# print("with yolo bounding boxes")

In [ ]:
# test_folder = "/content/drive/MyDrive/image_captioning/datasets/coco2014/val2014"  # or test2014 if you have it


# img_name_test = sorted([
#     os.path.join(test_folder, f)
#     for f in os.listdir(test_folder)
#     if f.lower().endswith(".jpg")
# ])

# # ✅ 3. Filter only those with extracted .npy features
# img_name_test = [
#     path for path in img_name_test
#     if os.path.exists(path + "_" + feature_extraction_model + ".npy")
# ]

In [ ]:
# import json, os
# from tqdm import tqdm
# import numpy as np
# import tensorflow as tf

# results = []

# # Directory where results.json should be saved
# results_path = os.path.join(save_dir, "results.json")

# # ✅ Ensure you have a loaded model (encoder + decoder)
# encoder_xcep.load_weights(os.path.join(save_dir, "encoder_xcep.weights.h5"))
# encoder_yolo.load_weights(os.path.join(save_dir, "encoder_yolo.weights.h5"))
# decoder.load_weights(os.path.join(save_dir, "decoder.weights.h5"))


# def evaluate_image(image_path):
#     """Generate a caption for one image."""
#     # Load precomputed features
#     img_tensor = np.load(image_path + "_" + feature_extraction_model + ".npy")
#     img_tensor = tf.convert_to_tensor(img_tensor)
#     img_tensor = tf.expand_dims(img_tensor, 0)

#     half = img_tensor.shape[1] // 2
#     xcep_feats = img_tensor[:, :half, :]
#     yolo_feats = img_tensor[:, half:, :]

#     xcep_encoded = encoder_xcep(xcep_feats)
#     yolo_encoded = encoder_yolo(yolo_feats)

#     result = []

#     for i in range(40):  # max caption length
#         predictions, hidden = decoder(dec_input, yolo_encoded, xcep_encoded, hidden)


#         predicted_id = tf.argmax(predictions[0]).numpy()
#         word = tokenizer.index_word.get(predicted_id, '')
#         if word == '<end>':
#             break
#         result.append(word)
#         dec_input = tf.expand_dims([predicted_id], 0)

#     return ' '.join(result)

# # ✅ Evaluate a subset or all test images
# test_images = img_name_test[:50]  # or full test set
# for img_path in tqdm(test_images):
#     if not os.path.exists(img_path + "_" + feature_extraction_model + ".npy"):
#         continue
#     predicted_caption = evaluate_image(img_path)
#     results.append({
#         "image_id": os.path.basename(img_path),
#         "predicted_caption": predicted_caption
#     })

# # ✅ Save results to Drive
# with open(results_path, "w") as f:
#     json.dump(results, f, indent=2)

# print(f"✅ Saved {len(results)} predictions to {results_path}")


In [ ]:
# # Load COCO annotations
# with open('/content/drive/MyDrive/image_captioning/datasets/coco2014/annotations/captions_val2014.json') as f:
#     val_data = json.load(f)

# results = []

# # Evaluate on a small subset first (say 500 images for speed)
# for ann in tqdm(val_data['annotations'][:500]):
#     img_id = ann['image_id']
#     img_path = f"/content/drive/MyDrive/image_captioning/datasets/coco2014/val2014/COCO_val2014_{img_id:012d}.jpg"

#     # Get predicted caption
#     pred_caption, _ = evaluate(img_path)
#     pred_caption = ' '.join(pred_caption).replace('<start>', '').replace('<end>', '').strip()

#     results.append({
#         "image_id": img_id,
#         "caption": pred_caption
#     })

# # Save predictions in COCO format
# with open("results.json", "w") as f:
#     json.dump(results, f)


In [ ]:
# print(results[:5])